[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# La especificación *JSON Schema*.

## El formato *JSON*.

*JSON* es el acrónimo de *Javascript Object Notation* y antes de las modificaciones publicadas en el estándar [*ECMAScript 2015*](https://www.ecma-international.org/ecma-262/6.0/), correspondía al modo en el que se definen objetos en *Javascript*. Sin embargo, la notación es tan simple y práctica que ha sido adoptada como un formato para la transmisión de datos y ha evolucionado de forma paralela a *Javascript*.

Para aprender más acerca de *JSON* es posible consultar la siguiente liga:

https://www.w3schools.com/js/js_json_intro.asp

### Tipos básicos de *JSON*.

* Cadenas de caracteres.
* Números.
* Boolenaos.
* Arreglos.

### Desventajas de *JSON*.

A diferencia de *XML* el formato *JSON* sólo cuenta con una definición muy limitadad de tipos de datos. Debido a esto, no es posible realizar validaciones avanzadas de los datos.

## Modelos y esquemas (*schemas*).

### Estado de un objeto.

En la programación orientada a objetos, es posible modelar de forma simple las características de objetos en un momento dado. A esto se le conoce como el estado de un objeto.

Cada dato que describe el estado de un objeto es ligado a un nombre o identificador al cual se le conoce como propiedad o atributo.

El modelo base de los objetos se conoce como *clase* y a los objetos emanados de dicha clase se les conoce como instancias.

### Esquemas.

Un esquema permite describir la forma en la que los datos serializados podrán ser validados.

## La especificación *JSON Schema*.

En función de la necesidad de crear una especificación que permita extender a *JSON* para representar estructuras de datos más complejas, se ha publicado [*JSON Schema*](https://json-schema.org/).

Dicha especificaci;on permite definir no sólo tipos, sino formatos, validadores y documentación a partir de *JSON*.

In [ ]:
!pip install jsonschema 

In [ ]:
import jsonschema

In [ ]:
esquema_alumno = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "nombre": {"type": "string",
                   "minLength":1,
                   },
        "primer_apellido": {"type": "string", 
                   "minLength":1,},
        "segundo_apellido": {"type": "string", 
                   "minLength":1,},
        "carrera": {"type": "string"},
        "semestre": {"type": "number",
                   "minimum": 1,
                   "maximum": 50,},
        "promedio": {"type": "number",
                   "minimum": 0,
                   "maximum": 10,},
        "al_corriente": {"type": "boolean"},
    },
    "required": ["nombre",  "primer_apellido", "carrera", "semestre",
                 "promedio", "al_corriente"]
}

In [ ]:
jsonschema.validate({'al_corriente': False,
                     'carrera': 'Arquitectura',
                     'nombre': 'Pedro', 
                     'primer_apellido': 'Solis', 
                     'promedio': 7.8, 
                     'semestre': 5,}, 
                    esquema_alumno)

In [ ]:
jsonschema.validate({'al_corriente': False,
                     'carrera': 'Arquitectura',
                     'nombre': 'Pedro', 
                     'primer_apellido': 'Solis', 
                     'promedio': 7.8, 
                     'semestre': 5,
                     'Género': 'F',},
                    esquema_alumno)

In [ ]:
jsonschema.validate({'Al Corriente': False, 
          'Carrera': 'Arquitectura', 
          'Nombre': 'Pedro', 
          'Primer Apellido': 'Solis', 
          'Promedio': 7.8, 
          'Semestre': -7,}, 
                    esquema_alumno)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>